In [1]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

import pandas as pd
import sqlite3
import hashlib
import time
from sqlalchemy import text, inspect, MetaData, Table, func
from sqlalchemy.orm import sessionmaker
from app.main import getTables, Base, Account, Transaction, TransactionFinal, Load
from app.helpers.readCsv import openOneCsv, getAccountHistory, setDateColumns
from app.helpers.sqlEngine import engine

pd.set_option('display.max_rows', 60)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 1000)


In [2]:
Session = sessionmaker(bind=engine)
session = Session()
session.connection().execute(text("DELETE FROM loads"))
session.connection().execute(text("DELETE FROM transactions_all"))
session.connection().commit()
session.close()

accounts = getTables(engine)

for index, account in accounts.iterrows():
    path = account['csvPath']
    print(path)
    
    Session = sessionmaker(bind=engine)
    session = Session()

    allFiles = getAccountHistory(path, account['id'], session)
    display(allFiles)
    allFiles.to_sql('transactions_all', con=session.connection(), if_exists='append', index=False, chunksize=10)
    session.connection().commit()
    session.close()


In [3]:

QUERY_FINAL_TRANSACTIONS = """
    SELECT
        sha256,
        postDate,
        transactionDate,
        accountId,
        amount,
        description,
        memo,
        details,
        type,
        category,
        balance,
        checkOrSlipNumber,
        count(*)
    FROM 
        transactions_all
    GROUP BY
        1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12
    ORDER BY
        postDate DESC
"""
df = setDateColumns(pd.read_sql(sql=text(QUERY_FINAL_TRANSACTIONS), con=engine.connect()))
#df = setDateColumns(df)
df.shape

In [8]:
Session = sessionmaker(bind=engine)
session = Session()
session.connection().execute(text("DELETE FROM transactions_final"))
session.commit()
session.close()

#session = Session()
#session.begin()

Session = sessionmaker(bind=engine)
session = Session()

query = session.query(
        Transaction.sha256,
        Transaction.postDate,
        Transaction.transactionDate,
        Transaction.accountId,
        Transaction.amount,
        Transaction.description,
        Transaction.memo,
        Transaction.details,
        Transaction.type,
        Transaction.category,
        Transaction.balance,
        Transaction.checkOrSlipNumber
    ).group_by(
        Transaction.sha256,
        Transaction.postDate,
        Transaction.transactionDate,
        Transaction.accountId,
        Transaction.amount,
        Transaction.description,
        Transaction.memo,
        Transaction.details,
        Transaction.type,
        Transaction.category,
        Transaction.balance,
        Transaction.checkOrSlipNumber,
    ).order_by(
        Transaction.postDate
    )

column_names = [ x['name'] for x in query.column_descriptions]

results = query.all()
print(results)
for idx, result in enumerate(results):
    row_data = dict(zip( column_names, result))
    transaction_final = TransactionFinal(**row_data)
    session.add(transaction_final)
    # print(row_data)
    session.commit()
    # if idx >= 2: break



#session.commit()
#session.connection().commit()
session.close()

In [15]:
Session = sessionmaker(bind=engine)
session = Session()

query = session.query(
        TransactionFinal.sha256,
        TransactionFinal.postDate,
        TransactionFinal.transactionDate,
        TransactionFinal.accountId,
        TransactionFinal.amount,
        TransactionFinal.description,
        TransactionFinal.memo,
        TransactionFinal.details,
        TransactionFinal.type,
        TransactionFinal.category,
        TransactionFinal.balance,
        TransactionFinal.checkOrSlipNumber
    ).order_by(
        TransactionFinal.postDate
    )

#results = query.all()

#for result in results:
#    print(result)
result = query.all()

df_final = pd.DataFrame(result, columns=column_names)
df_final = setDateColumns(df_final)
session.close()


In [4]:
def describe(search, pdf):
    sdf = pdf[pdf['description'].str.contains(search, case=False)]
    display(sdf)
    amountSum = sdf['amount'].sum()
    numTransactions = sdf.shape[0]
    if numTransactions <= 0: return
    print(f"total number of transactions= {numTransactions}")
    print(f"sum= {amountSum}")
    print(f"avg cost per transaction= {round(amountSum/numTransactions, 2)}")
    latestDate = sdf.iloc[0]['postDate']
    startingDate = sdf.iloc[-1]['postDate']
    totalDays = (latestDate - startingDate).days
    print(f"dates= {startingDate} to {latestDate}, total days={totalDays}")
    avgDaysBetweenTransaction = totalDays / numTransactions
    print(f"avg days between transaction= {avgDaysBetweenTransaction}")
    amountSum
    costPerMonth = amountSum / (totalDays / 30.437)
    print(f"avg cost per month= {costPerMonth}")

In [18]:
describe('BLICK', df_final)

In [22]:
describe('publix', df)

In [23]:
describe('publix', df_final)

In [7]:
describe('publix', df)